In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [211]:
#helper
def numConcat(num1, num2):
        num1 = str(num1)
        num2 = str(num2)
        num1 += num2
        return int(num1)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/kalinakalina/DEDA_Cryptopunks/main/transaction.csv', header = None, index_col = 0)
df.columns = ['type', 'from', 'to', 'date', 'punk_id', 'amount_in_eth', 'amount_in_dol']
df = df[ ['punk_id'] + [ col for col in df.columns if col != 'punk_id' ] ]

In [3]:
df['punk_id'] = df['punk_id'].astype('category')
df['type'] = df['type'].astype('category')
df['from'] = df['from'].astype(str)
df['to'] = df['to'].astype(str)
df['date'] = pd.to_datetime(df['date'], format = '%Y.%m.%d')

In [4]:
#Fix amount values in eth and dollar
df['amount_in_eth'] = df['amount_in_eth'].astype(str)
df['amount_in_eth'] = df['amount_in_eth'].str.replace('Y', '')
df['amount_in_eth'] = df['amount_in_eth'].str.replace('Z', '')
df['amount_in_eth'] = df['amount_in_eth'].str.replace('M', '')
df['amount_in_eth'] = df['amount_in_eth'].str.replace('T', '')
df['amount_in_eth'] = df['amount_in_eth'].str.replace('B', '')
df['amount_in_eth'] = df['amount_in_eth'].replace(',', '', regex=True)
df['amount_in_eth'] = df['amount_in_eth'].astype(float)

df['amount_in_dol'] = df['amount_in_dol'].astype(str)
df['amount_in_dol'] = df['amount_in_dol'].str.replace('Y', '')
df['amount_in_dol'] = df['amount_in_dol'].str.replace('B', '')
df['amount_in_dol'] = df['amount_in_dol'].str.replace('T', '')
df['amount_in_dol'] = df['amount_in_dol'].str.replace('P', '')
df['amount_in_dol'] = df['amount_in_dol'].str.replace('Z', '')
df['amount_in_dol'] = df['amount_in_dol'].replace('<', '', regex=True)
df['amount_in_dol'] = df['amount_in_dol'].replace(',', '', regex=True)
df['amount_in_dol'] = df['amount_in_dol'].astype(float)

In [5]:
df['amount_in_eth'] = df['amount_in_eth'].fillna(0)
df['amount_in_dol'] = df['amount_in_dol'].fillna(0)

#EDA

In [220]:
df_sold = df.copy()
df_sold = df_sold[df_sold['type'].str.contains("Sold")] 
df_sold.drop(columns=['type', 'from','to', 'amount_in_eth'], inplace=True)
df_sold

,punk_id,date,amount_in_dol
0,,,
16,0,2018-11-30,2822.0
24,0,2017-07-07,386.0
31,0,2017-06-23,320.0
1,1,2020-11-30,36305.0
27,1,2019-04-06,5155.0
...,...,...,...
0,9990,2021-03-20,44981.0
6,9997,2021-02-08,169770.0
20,9997,2020-09-27,21062.0


# Dummy Encoding

In [221]:
df_sold['year'] = pd.DatetimeIndex(df_sold['date']).year
df_sold['month'] = pd.DatetimeIndex(df_sold['date']).month
df_sold['monthyear'] = df_sold.apply(lambda row: numConcat(row.month, row.year), axis=1)
df_sold.drop(columns= ['month', 'year'], inplace=True)
df_sold.head()

,punk_id,date,amount_in_dol,monthyear
0,,,,
16,0,2018-11-30,2822.0,112018
24,0,2017-07-07,386.0,72017
31,0,2017-06-23,320.0,62017
1,1,2020-11-30,36305.0,112020
27,1,2019-04-06,5155.0,42019


In [222]:
df_sold['monthyear']=df_sold['monthyear'].astype('category')
df_sold.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13531 entries, 16 to 7
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   punk_id        13531 non-null  category      
 1   date           13531 non-null  datetime64[ns]
 2   amount_in_dol  13531 non-null  float64       
 3   monthyear      13531 non-null  category      
dtypes: category(2), datetime64[ns](1), float64(1)
memory usage: 756.5 KB


In [224]:
dfsd= pd.get_dummies(df_sold, columns=['monthyear'])
dfsd.head()

,punk_id,date,amount_in_dol,monthyear_12018,monthyear_12019,monthyear_12020,monthyear_12021,monthyear_22018,monthyear_22019,monthyear_22020,monthyear_22021,monthyear_32018,monthyear_32019,monthyear_32020,monthyear_32021,monthyear_42018,monthyear_42019,monthyear_42020,monthyear_42021,monthyear_52018,monthyear_52019,monthyear_52020,monthyear_52021,monthyear_62017,monthyear_62018,monthyear_62019,monthyear_62020,monthyear_72017,monthyear_72018,monthyear_72019,monthyear_72020,monthyear_82017,monthyear_82018,monthyear_82019,monthyear_82020,monthyear_92017,monthyear_92018,monthyear_92019,monthyear_92020,monthyear_102017,monthyear_102018,monthyear_102019,monthyear_102020,monthyear_112017,monthyear_112018,monthyear_112019,monthyear_112020,monthyear_122017,monthyear_122018,monthyear_122019,monthyear_122020
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16,0,2018-11-30,2822.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24,0,2017-07-07,386.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31,0,2017-06-23,320.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2020-11-30,36305.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
27,1,2019-04-06,5155.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [225]:
dfsd.to_csv('sales_time_dummie_encoded.csv', mode='w')